<a href="https://colab.research.google.com/github/miguelvelascop/PFG_Miguel_Velasco_Puig/blob/main/Preprocesado_Dataset_Depresjon.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Se importan librerías para manejar vectores y matrices (numpy) y manejar tablas (panda) más facilmente, y os, que permite moverse por el sistema de archivos más facilmente(en este caso por GoogleDrive).

In [ ]:
import numpy as np
import pandas as pd
import os

También se importan las funciones de las librería de colab que permiten cargar archivos.

In [ ]:
from google.colab import files, drive

Se van creando dataframes con cada csv cargado, se le aplican las funciones para obtener la media, desviación típica, el mínimo y el máximo, se pasan a arrays numpy para poder ir uniendolos y conseguir al final una matriz numpy con todos los datos necesarios, y las etiquetas añadidas. Se repite el proceso tanto con archivos de depresión como no depresión, y al final se concatenan ambas matrices.

In [ ]:
df = pd.DataFrame()
#Se cargarn tres listas con todos los archivos csv del dataset (una para no depresión, otra para depresión y otro con datos sobre los voluntarios)
ruta_Depresion='/content/drive/MyDrive/TFG/datasets/depresjon/condition'
ruta_noDepresion='/content/drive/MyDrive/TFG/datasets/depresjon/control'
ruta_Scores='/content/drive/MyDrive/TFG/datasets/depresjon/scores.csv'
archivos_De = os.listdir(ruta_Depresion)
archivos_noDe = os.listdir(ruta_noDepresion)
scores=pd.read_csv(ruta_Scores,  sep=',',  comment='#')

#Se declara una función para filtrar los datos incompletos posteriormente
dayIsComplete = lambda x: x['activity'].count() == 1440

#Agrupar todos los datos de depresión en un pandas
#Prepocesado datos depresión

cont=0;
for i in archivos_De:
  aux=pd.read_csv(ruta_Depresion+'/'+i,  sep=',',  comment='#')
  aux['timestamp'] = pd.to_datetime(aux['timestamp'])

  aux = aux.groupby('date').filter(dayIsComplete)

  media = aux.groupby('date')['activity'].mean().to_numpy()
  matriz=media

  desv = aux.groupby('date')['activity'].std().to_numpy()
  matriz = np.append([matriz],[desv],axis=0)

  min = aux.groupby('date')['activity'].min().to_numpy()
  matriz2 = min

  max = aux.groupby('date')['activity'].max().to_numpy()
  matriz2 = np.append([matriz2],[max],axis=0)

  matriz = np.insert(matriz, matriz.shape[0], matriz2, 0)
  total = aux.groupby('date')['activity'].sum().to_numpy()
  matriz = np.append(matriz,[total],axis=0)

  matriz=np.transpose(matriz)

  #Añadir columna con genero
  selected_score = scores[scores['number'] == i.split('.')[0]]
  gender = selected_score['gender'].values[0]
  n_filas = matriz.shape[0]
  columna_genero = np.full((n_filas, 1), gender)
  matriz = np.hstack((matriz, columna_genero))

  if(cont==0):
    matrizdefD=matriz
  else:
    matrizdefD=np.concatenate((matrizdefD, matriz))
  cont=cont+1

mediaAuxD = matrizdefD[:,4].mean()
desED = matrizdefD[:,4].std()

#Añadir fila de etiqueta (0, Depresión)
a = np.zeros((1,matrizdefD.shape[0]))
matrizdefD = np.insert(matrizdefD, 6, a, 1)

print("Depession data size:")
print(matrizdefD.shape)

#######################################
#Prepocesado datos no depresión

cont=0
for i in archivos_noDe:
  aux=pd.read_csv(ruta_noDepresion+'/'+i,  sep=',',  comment='#')
  aux['timestamp'] = pd.to_datetime(aux['timestamp'])

  aux = aux.groupby('date').filter(dayIsComplete)

  media = aux.groupby('date')['activity'].mean().to_numpy()
  matriz=media

  desv = aux.groupby('date')['activity'].std().to_numpy()
  matriz = np.append([matriz],[desv],axis=0)

  min = aux.groupby('date')['activity'].min().to_numpy()
  matriz2 = min

  max = aux.groupby('date')['activity'].max().to_numpy()
  matriz2 = np.append([matriz2],[max],axis=0)

  matriz = np.insert(matriz, matriz.shape[0], matriz2, 0)
  total = aux.groupby('date')['activity'].sum().to_numpy()
  matriz = np.append(matriz,[total],axis=0)

  matriz=np.transpose(matriz)

  #Añadir columna con genero
  selected_score = scores[scores['number'] == i.split('.')[0]]
  gender = selected_score['gender'].values[0]
  n_filas = matriz.shape[0]
  columna_genero = np.full((n_filas, 1), gender)
  matriz = np.hstack((matriz, columna_genero))


  if(cont==0):
    matrizdefND=matriz
  else:
    matrizdefND=np.concatenate((matrizdefND, matriz))
  cont=cont+1

mediaAuxND = matrizdefND[:,4].mean()
desEND = matrizdefND[:,4].std()

#Añadir fila de etiqueta (0, Depresión)
a = np.ones((1,matrizdefND.shape[0]))
matrizdefND = np.insert(matrizdefND, 6, a, 1)

print("No depession data size:")
print(matrizdefND.shape)
#Concatenar matrizes de depresión y no depresión

matrizDef=np.concatenate((matrizdefD, matrizdefND))

print("Final data size:")
print(matrizDef.shape)



Ahora se pasa la matriz numpy con los datos obtenidos a un dataframe pandas

In [ ]:
dataFrame= pd.DataFrame(matrizDef, columns = ['media','desviacion_tipica','minimo','maximo','total','genero','etiqueta'])

Por último, se importa el dataframe a un csv y se descarga

In [ ]:
#Con index=False se quitan los índices
dataFrame.to_csv('Dataset_Depresjon_procesado.csv', index=False)
files.download('Dataset_Depresjon_procesado.csv')